In [1]:
### Working With Tools
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper , DuckDuckGoSearchAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun , DuckDuckGoSearchResults

## Arxiv and wikipedia Tools
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=2000)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=2000)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

# wrapper = DuckDuckGoSearchAPIWrapper(time="d", max_results=10)
# searsh = DuckDuckGoSearchResults(api_wrapper=wrapper)

searsh = DuckDuckGoSearchResults(name="Search") 

In [2]:
tools = [wiki , arxiv , searsh]

In [3]:
# custom tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=2000 , chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(documents , embeddings)
retrever = vectorstore.as_retriever()
retrever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f0d82e81b80>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool

In [8]:
retriever_tool=create_retriever_tool(retrever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name


'langsmith-search'

In [9]:
tools = [wiki , arxiv , searsh ,retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/mahmoud/anaconda3/envs/lunfgraph/lib/python3.9/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=2000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=2000)),
 DuckDuckGoSearchResults(name='Search', api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='api', source='text')),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_docume

In [11]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Access keys
groq_api_key = os.getenv('GROQ_API_KEY')

In [12]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key ,model_name="Llama3-8b-8192")

In [13]:
llm.invoke("Hello")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.021666667, 'prompt_time': 0.001758703, 'queue_time': 0.017523145, 'total_time': 0.02342537}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-a9d19162-7bd7-4c67-8f09-144aa7dcb172-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})

# prompt template

In [14]:
from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/home/mahmoud/anaconda3/envs/lunfgraph/lib/python3.9/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

# Agents 

In [15]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm ,tools , prompt)

agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [16]:
## agent Executer
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.agents import AgentExecutor

memory = ConversationBufferMemory(k=3)  # Default memory setup


agent = AgentExecutor(agent=agent, tools=tools, memory=memory,verbose=True)

agent

/tmp/ipykernel_46087/563566781.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(k=3)  # Default memory setup


AgentExecutor(memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[])), verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AI

In [17]:
agent.invoke({"input":"What is the capital of France?"})  



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Capital of France'}`


Page: List of capitals of France
Summary: This is a chronological list of capitals of France. The capital of France has been Paris since its liberation in 1944.


Invoking: `wikipedia` with `{'query': 'Paris'}`


Page: Paris
Summary: Paris (French pronunciation: [paʁi] ) is the capital and largest city of France. With an estimated population of 2,102,650 residents in January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-most populous city in the European Union, the ninth-most populous city in Europe and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. Because of its leading role in the arts and sciences and its early adaptation of extensive street lighting, it became known as the City of Light in the 19th century.
T

{'input': 'What is the capital of France?',
 'history': '',
 'output': 'Paris is the capital and largest city of France.'}

In [18]:
agent.invoke({"input":"What is Machine Learning?"})  



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.
Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning. 
From a theoretical viewpoint, p

{'input': 'What is Machine Learning?',
 'history': 'Human: What is the capital of France?\nAI: Paris is the capital and largest city of France.',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence that enables algorithms to learn from data and make predictions or decisions without being explicitly programmed. It involves the development and study of statistical algorithms that can generalize to new, unseen data. ML has many applications in various fields, including natural language processing, computer vision, speech recognition, and more. The field of ML is built on foundations of statistics and mathematical optimization, and data mining is a related field that focuses on exploratory data analysis.'}

In [19]:
agent.invoke({"input":"What is gym mtsim ?"})  



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Gym MTSim'}`


No good Wikipedia Search Result was found
Invoking: `arxiv` with `{'query': 'Gym MTSim'}`


Published: 2022-03-09
Title: Gym-saturation: an OpenAI Gym environment for saturation provers
Authors: Boris Shminke
Summary: `gym-saturation` is an OpenAI Gym environment for reinforcement learning (RL)
agents capable of proving theorems. Currently, only theorems written in a
formal language of the Thousands of Problems for Theorem Provers (TPTP) library
in clausal normal form (CNF) are supported. `gym-saturation` implements the
'given clause' algorithm (similar to the one used in Vampire and E Prover).
Being written in Python, `gym-saturation` was inspired by PyRes. In contrast to
the monolithic architecture of a typical Automated Theorem Prover (ATP),
`gym-saturation` gives different agents opportunities to select clauses
themselves and train from their experience. Combined with a particular agent,


{'input': 'What is gym mtsim ?',
 'history': 'Human: What is the capital of France?\nAI: Paris is the capital and largest city of France.\nHuman: What is Machine Learning?\nAI: Machine learning (ML) is a field of study in artificial intelligence that enables algorithms to learn from data and make predictions or decisions without being explicitly programmed. It involves the development and study of statistical algorithms that can generalize to new, unseen data. ML has many applications in various fields, including natural language processing, computer vision, speech recognition, and more. The field of ML is built on foundations of statistics and mathematical optimization, and data mining is a related field that focuses on exploratory data analysis.',
 'output': 'It seems like the user is asking about OpenAI Gym. Since we already have information about Gym MTSim, I can provide a direct response.\n\nOpenAI Gym is an open-source software framework that allows researchers and developers to 